In [1]:
from database import ResstockDatabase, SQLiteDatabase
from simulate import CityLearnSimulator

In [2]:
database_filepath = 'database.sql'
resstock_database = ResstockDatabase(filepath=database_filepath)

# Build Database
***

In [3]:
# filters = {
#     'in.geometry_building_type_recs':['Single-Family Detached'],
#     'in.vacancy_status':['Occupied'],
#     'in.resstock_county_id':['TX, Travis County']
# }
# kwargs = {
#     'dataset_type':'resstock',
#     'weather_data':'tmy3',
#     'year_of_publication':2021,
#     'release':1,
#     'filters':filters
# }
# resstock_database.build(overwrite=True)
# resstock_database.insert_dataset(**kwargs)

# E+ Simulation
***

In [126]:
idd_filepath = '/Applications/EnergyPlus-9-6-0/PreProcess/IDFVersionUpdater/V9-6-0-Energy+.idd'
bldg_id = '40'
sim_data = resstock_database.query_table(f"""
SELECT
    metadata_id,
    osm,
    epw,
    ddy
FROM building_energy_performance_simulation_input
WHERE bldg_id = {bldg_id}
""")
metadata_id, osm, epw, ddy = sim_data.to_records(index=False)[0]
schedule = resstock_database.query_table(f"""
SELECT
    *
FROM schedule
WHERE metadata_id = {metadata_id}
""")
schedule = schedule.drop(columns=['metadata_id','day','hour','minute',])
schedule.to_csv('schedules.csv',index=False)
idf = CityLearnSimulator.osm_to_idf(osm)

with open('model.idf','w') as f:
    f.write(idf)

output_directory = f'{bldg_id}_simulation'
simulator = CityLearnSimulator(idd_filepath,idf,epw,ddy=ddy,simulation_id=bldg_id,output_directory=output_directory)
simulator.simulate()


/Applications/EnergyPlus-9-6-0/energyplus --weather /Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/CityLearn/nrel_xstock/nrel_xstock/40_simulation/weather.epw --output-directory /Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/CityLearn/nrel_xstock/nrel_xstock/40_simulation --idd /Applications/EnergyPlus-9-6-0/PreProcess/IDFVersionUpdater/V9-6-0-Energy+.idd --expandobjects --output-prefix 40 --output-suffix C /Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/CityLearn/nrel_xstock/nrel_xstock/in.idf

ExpandObjects Started.
No expanded file generated.
ExpandObjects Finished. Time:     0.067
EnergyPlus Starting
EnergyPlus, Version 9.6.0-f420c06a69, YMD=2021.12.10 17:54


Could not find platform independent libraries <prefix>
Could not find platform dependent libraries <exec_prefix>
Consider setting $PYTHONHOME to <prefix>[:<exec_prefix>]


Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Starting Simulation at 01/01/2018 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=01/21/2018
Continuing Simulation at 01/21/2018 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=02/10/2018
Continuing Simulation at 02/10/2018 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=03/02/2018
Continuing Simulation at 03/02/2018 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=03/22/2018
Continuing Simulation at 03/22/2018 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=04/11/2018
Continuing Simulation at 04/11/2018 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=05/01/2018
Continuing Simulation at 05/01/2018 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=05/21/2018
Con

EnergyPlus Completed Successfully.


In [132]:
database = SQLiteDatabase('40_simulation/40.sql')
with open('misc/citylearn_simulation_results.sql','r') as f:
    query = f.read()

data = database.query_table(query)
display(data)

,Month,Hour,Day Type,Daylight Savings Status,Indoor Temperature (C),Average Unmet Cooling Setpoint Difference (C),Average Unmet Heating Setpoint Difference (C),Indoor Relative Humidity (%),Equipment Electric Power (kWh),DHW Heating (kWh),Cooling Load (kWh),Heating Load (kWh)
0,1,1,1,0,22.250907,1.637982e+00,1.695352e+00,29.873259,0.389051,0.841179,11.370526,-11.768633
1,1,2,1,0,21.925969,1.962920e+00,1.370413e+00,30.489844,0.273632,0.772949,9.558740,-6.673276
2,1,3,1,0,21.567766,2.321123e+00,1.012210e+00,31.212749,0.206945,1.317195,13.856985,-6.042505
3,1,4,1,0,21.272736,2.616153e+00,7.171801e-01,31.801637,0.204379,0.761527,12.771067,-3.500902
4,1,5,1,0,20.886832,3.002057e+00,3.312768e-01,32.607279,0.235162,0.711725,14.093287,-1.555200
5,1,6,1,0,20.583464,3.305425e+00,2.790856e-02,33.296981,0.425893,0.985702,15.944190,0.074395
6,1,7,1,0,20.555652,3.333237e+00,9.650549e-05,33.449050,0.877691,1.143227,17.041115,0.765935
7,1,8,1,0,23.820407,6.848142e-02,6.848142e-02,27.487319,1.171252,1.318820,10.177393,10.177393
8,1,9,1,0,23.888095,7.936840e-04,7.936840e-04,27.431792,1.280598,1.318820,15.633696,15.633696
9,1,10,1,0,23.888890,8.904330e-07,8.904330e-07,27.424136,1.288687,1.318820,13.087301,13.087301


In [125]:
database.query_table("SELECT * FROM ReportData WHERE ReportDataDictionaryIndex IN (954, 955, 956)").describe()

,ReportDataIndex,TimeIndex,ReportDataDictionaryIndex,Value
count,26280.000000,26280.000000,26280.000000,26280.000000
mean,254044.000000,4380.500000,955.000000,-3428.983956
std,146672.852037,2528.842276,0.816512,11297.876154
min,32.000000,1.000000,954.000000,-84755.429342
25%,127038.000000,2190.750000,954.000000,-8167.924011
50%,254044.000000,4380.500000,955.000000,0.000000
75%,381050.000000,6570.250000,956.000000,0.000000
max,508056.000000,8760.000000,956.000000,97073.084034


In [129]:
database.get_table('ReportDataDictionary')

,ReportDataDictionaryIndex,IsMeter,Type,IndexGroup,TimestepType,KeyValue,Name,ReportingFrequency,ScheduleName,Units
0,10,0,Sum,Zone,Zone,RES LIGHTING EXTERIOR,Exterior Lights Electricity Energy,Hourly,None,J
1,48,0,Sum,Zone,Zone,RES LIGHTING GARAGE GARAGE SPACE,Lights Electricity Energy,Hourly,None,J
2,157,0,Sum,Zone,Zone,RES LIGHTING INTERIOR LIVING SPACE,Lights Electricity Energy,Hourly,None,J
3,218,0,Sum,Zone,Zone,RES LIGHTING INTERIOR LIVING SPACE|STORY 2,Lights Electricity Energy,Hourly,None,J
4,255,0,Sum,Zone,Zone,RES EXTRA REFRIG,Electric Equipment Electricity Energy,Hourly,None,J
5,328,0,Sum,Zone,Zone,RES CD ELECTRIC,Electric Equipment Electricity Energy,Hourly,None,J
6,377,0,Sum,Zone,Zone,RES CEIL FAN,Electric Equipment Electricity Energy,Hourly,None,J
7,414,0,Sum,Zone,Zone,RES CW,Electric Equipment Electricity Energy,Hourly,None,J
8,451,0,Sum,Zone,Zone,RES INFIL_1 BATH FAN,Electric Equipment Electricity Energy,Hourly,None,J
9,488,0,Sum,Zone,Zone,RES INFIL_1 HOUSE FAN,Electric Equipment Electricity Energy,Hourly,None,J
